In [ ]:
!pip install scipy
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install pandoc

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import seaborn as sns

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
CENTURY_PATTERN = re.compile("Century", re.IGNORECASE)
JM_PATTERN = re.compile("(JM)|(John[ _]+Marshall)", re.IGNORECASE)
MAYO_PATTERN = re.compile("Mayo", re.IGNORECASE)

TEAM_MAP = {
 "CENTURY" : "CENTURY & JM",
 "JM" : "CENTURY & JM",
 "MAYO" : "MAYO"
}

In [ ]:
paths = [
    "/home/ntl/notebooks/vie_grade/Century_board_results.csv",
    "/home/ntl/notebooks/vie_grade/JM_board_results.csv",
    "/home/ntl/notebooks/vie_grade/Mayo_board_results.csv"
]

dfs = [pd.read_csv(p) for p in paths]

df = pd.concat(dfs)

In [ ]:
def get_school_from_jar_path(jar_path: str) -> str:
    if CENTURY_PATTERN.match(jar_path):
        return "CENTURY"
    elif JM_PATTERN.match(jar_path):
        return "JM"
    elif MAYO_PATTERN.match(jar_path):
        return "MAYO"
    else:
        raise ValueError(f"{jar_path} does not begin with a school")
    
    


In [ ]:
df["School"] = df["Jar Path"].apply(get_school_from_jar_path)
df["Team"] = df["School"].apply(lambda path: TEAM_MAP.get(get_school_from_jar_path(path)))

In [ ]:
#df = df.loc[~df["Board Name"].isin(['Level_3_4.csv', 'Level_3_9.csv', 'Surprise_2.csv', 'Surprise_3.csv']), :]


df["Team"] = df["Team"].astype("category")
df["School"] = df["School"].astype("category")
df["Board Name"] = df["Board Name"].astype("category")

In [ ]:
df["Player Rank on Board"] = df.groupby(by=["Board Name"])["Score"].rank("dense", ascending=False)

In [ ]:
at_rank = df.groupby(by=["Board Name", "Player Rank on Board"], as_index=True).size()

In [ ]:
def calc_score(df_row, at_rank):
    points = {}
    points[1] = 3
    points[2] = 2
    points[3] = 1
    
    board_name = df_row["Board Name"]
    rank = df_row["Player Rank on Board"]
    
    ties = {}
    ties[1] = at_rank.get((board_name, 1), 0)
    ties[2] = at_rank.get((board_name, 2), 0)
    ties[3] = at_rank.get((board_name, 3), 0)
    
    if ties[1] > 1:
        points[1] = points[1] + points[2]
        points[2] = points[3]
        points[3] = 0
    if ties[1] > 2:
        points[1] = points[1] + points[2]
        points[2] = 0
    if ties[2] > 1:
        points[2] = points[2] + points[3]
        points[3] = 0
        
    score = points.get(rank, 0)
    if score > 0 and ties.get(rank, 0) > 0:
        return score / ties.get(rank)
    
    return score
            

In [ ]:
df["Points"] = df.apply(calc_score, args=(at_rank,), axis='columns')

# Board Winners

In [ ]:
board_winners = df.sort_values(["Board Name", "Player Rank on Board"], ascending=True).set_index(["Board Name", "Player Rank on Board"])
board_winners.loc[board_winners["Points"] > 0, ["Student Name", "School", "Team", "Score", "Points"]]

# Number of Players with same rank by Board

In [ ]:
df["Player Rank on Board"] = df["Player Rank on Board"].apply(lambda r: int(r))
ties = df.loc[df["Player Rank on Board"] <= 3].groupby(by=["Board Name", "Player Rank on Board"]).size().to_frame("Players at rank").unstack()
ties.sort_values([("Players at rank", 1)], ascending=False)

In [ ]:
ties[("Players at rank", 1)].plot.bar(figsize=(20,5))

# Scores by Player

In [ ]:
df.sort_values(["Student Name", "Board Name"], ascending=True).set_index(["Student Name", "Board Name"])

# Top individual scores (All boards)

In [ ]:
top_idv_scores = pd.DataFrame(df.groupby(by=["Student Name", "School"])[["Score"]].sum()).sort_values(["Score"], ascending=False)
top_idv_scores.head(15)

In [ ]:
top_idv_scores.reset_index().groupby(by="School")["Score"].std()

# Top Points

In [ ]:
tp = pd.DataFrame(df.groupby(by=["Student Name", "School"])[["Points"]].sum()).sort_values(["Points"], ascending=False)
tp.loc[tp["Points"] > 0, :]

# Submissions by school

In [ ]:
df.groupby(by=["School"])["Jar Path"].unique().apply(lambda l: len(l))

# Team Scores

In [ ]:
df.groupby(by=["Team"])["Points"].sum()

In [ ]:
df.groupby(by=["Team"])["Points"].sum().plot.bar(title="Points by Team")

In [ ]:
df.groupby(by=["School"])["Points"].sum().plot.bar(title="Points by school")

# Scores by board

In [ ]:
fig, ax = plt.subplots(figsize=(20,15))
ax.tick_params(axis='x', labelrotation=90)
sns.scatterplot(ax=ax, x=df["Board Name"], y=df["Score"], s=50)

# Variation in Scores

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
ax.tick_params(axis='x', labelrotation=90)
sns.boxplot(ax=ax, x=df["Board Name"], y=df["Score"], width=.5)

# Errors

In [ ]:
ax = df.groupby(by=["Failure"])["Failure"].count().plot.pie(title="Types of exceptions")
ax.set_ylabel('')

# Errors by School

In [ ]:
df.groupby(by="School")["Failure"].value_counts()

# Errors by Student / Board

In [ ]:
errors = df.loc[pd.notnull(df["Failure"])].sort_values(["Student Name", "Board Name"], ascending=True)
errors.set_index(["Student Name", "Board Name"], inplace=True)
errors.loc[:, ["School", "Failure", "Jar Path"]]

# Cumulative score by board

In [ ]:
df2 = df.sort_values(["Board Name", "Points"])
df2 = df2.groupby(by=["Team", "Board Name"])["Points"].sum().to_frame().reset_index().sort_values(["Board Name"]).reset_index(drop=True)
df2['cs'] = df2.groupby(by=["Team"])["Points"].cumsum()
df3 = df2.set_index(["Board Name"]).groupby(["Team"])["cs"]
df3.plot(rot=90, figsize=(5,5))


# Points by board

In [ ]:
df2 = df.groupby(by=['School', 'Board Name'])['Points'].sum().to_frame().reset_index()
#display(df2)
display(df2.pivot_table(df2, index='Board Name', columns=['School']))
df2.pivot_table(df2, index='Board Name', columns=['School']).plot(kind='bar', ylim=(0,7), figsize=(10,10), stacked=True)